In [1]:
from h5Dataset import h5Dataset
from modules import MPNNLayer, MPNNTokenizer, SelfAttentionEncoder, PredictionHead
from model import MPNNTransformerModel

import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [3]:
def main():
   
    torch.manual_seed(0)
    #Using GPU 3, check if available before running
    device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

    # Load Dataset
    h5_path = "../data/samples/100samples.h5"  # <- CHANGE THIS
    ds = h5Dataset(h5_path)

    loader = DataLoader(ds, batch_size=100)

    # take feature dims from one sample
    sample0 = ds[0]
    node_in_dim = sample0.x.shape[-1]
    edge_in_dim = sample0.edge_attr.shape[-1]

    # Build model
    model = MPNNTransformerModel(
        node_in_dim=node_in_dim,
        edge_in_dim=edge_in_dim,
        num_output_sources=1,  
    ).to(device)

    # Optimizer, loss, scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=0.0)

    scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=25,   # every 10 epochs
        gamma=0.8      # multiply LR by 0.8
    )

    # 6) Train loop (overfit)
    num_epochs = 500  # usually enough to see memorization
    for epoch in range(1, num_epochs + 1):

        model.train()
        running_loss = 0.0

        for data in loader:
            data = data.to(device)

            pred = model.forward_from_data(data)  # expected shape: [I, 2] with I=1
            
            target = data.y

            loss = F.mse_loss(pred, target)

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            

        scheduler.step()

        avg_loss = running_loss / len(loader)
        
        #print occasionally
        if epoch % 25 == 0 or epoch == 1:
            print(f"Epoch {epoch:4d} | loss = {avg_loss:.6f}")

        if avg_loss < 1e-6:
            print(f"Early stopping at epoch {epoch} with loss {avg_loss:.6f}")
            break

    #Quick evaluation on the same samples

    model.eval()
    print("\nPredictions vs targets:")
    with torch.no_grad():
        for i, data in enumerate(ds):
            data = data.to(device)
            pred = model.forward_from_data(data)  # [1,2]
            target = data.y
            if target.dim() == 1:
                target = target.unsqueeze(0)
            if target.shape[-1] == 3:
                target = target[:, :2]

            print(f"[{i:02d}] pred={pred.squeeze(0).cpu().numpy()}  target={target.squeeze(0).cpu().numpy()}")

if __name__ == "__main__":
    main()        


Epoch    1 | loss = 0.038248
Epoch   25 | loss = 0.027090
Epoch   50 | loss = 0.026396


KeyboardInterrupt: 